In [2]:
pip install tensorflow==1.15.0

     |████████████████████████████████| 124.0MB 99kB/s  eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/8e/b0/6e68b980615289234149903613f1d6af09d656f5d9bdaa69411aa980b86c/protobuf-3.12.0-cp37-cp37m-macosx_10_9_x86_64.whl
     |████████████████████████████████| 2.8MB 2.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 4.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/1a/53/9243c600e047bd4c3df9e69cfabc1e8004a82cac2e0c484580a78a94ba2a/absl-py-0.9.0.tar.gz
     |████████████████████████████████| 71kB 3.6MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 512kB 1.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4e/35/11749bf99b2d4e3cceb4d55ca22590b0d7c2c62b9de38ac4a4a7f4687421/gast-0.2.2.tar.gz
     |█████████████

In [13]:
!brew update -qq && brew install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev


Already up-to-date.
Usage: brew install [options] formula

Install formula. Additional options specific to formula may be appended to
the command.

Unless HOMEBREW_NO_INSTALL_CLEANUP is set, brew cleanup will then be run for
the installed formulae or, every 30 days, for all formulae.

    -d, --debug                      If brewing fails, open an interactive
                                     debugging session with access to IRB or a
                                     shell inside the temporary build directory.
        --env                        If std is passed, use the standard build
                                     environment instead of superenv. If super
                                     is passed, use superenv even if the formula
                                     specifies the standard build environment.
        --ignore-dependencies        An unsupported Homebrew development flag to
                                     skip installing any dependencies of any
    

In [16]:
!pip install fluidsynth

  Created wheel for fluidsynth: filename=fluidsynth-0.2-cp37-none-any.whl size=4515 sha256=69c84da0198f79256a36b449dae756c8640857ca9ccf2ec89bc1aea78191c0fc
  Stored in directory: /Users/mendeloster/Library/Caches/pip/wheels/2d/70/4d/ffe2589bbfe9c268780570a6875583fbf59547004459e2c3c3
Successfully built fluidsynth


In [17]:
#@test {"output": "ignore"}
# %tensorflow_version 1.x

print('Installing dependencies...')
# !apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
    if lib == 'fluidsynth':
        return 'libfluidsynth.so.1'
    else:
        return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Installing dependencies...
Importing libraries and defining some helper functions...


RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
# Model options. Change these to get different generated sequences! 

input_sequence = keili # change this to teapot if you want
num_steps = 500 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

mm.plot_sequence(sequence)
mm.play_sequence(sequence, synth=mm.fluidsynth)